In [6]:
import pandas as pd
import numpy as np

# Dataset
marketing = pd.read_csv('bank_marketing.csv')

# Client
client = marketing[['client_id','age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
companies = marketing[['client_id', 'cons_price_idx', 'euribor_three_months']]

column_replace = ['job', 'education'] 
for col in column_replace:
    client[col] = client[col].str.replace(".", "_")
    if col == 'education':
        client[col] = client[col].replace("unknown", np.NaN)
        
for col in ['credit_default', 'mortgage']:
    client[col] = client[col].map({"no": 0, "unknown": 0, "yes": 1})
    client[col] = client[col].astype(bool)

# Campaign
campaign = marketing[['client_id','number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success': 1, 'failure': 0, 'nonexistent': 0})
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'no': 0, 'yes': 1})

print(campaign['previous_outcome'])

for col in ['previous_outcome', 'campaign_outcome']:
    campaign[col] = campaign[col].astype(bool)
    
marketing['year'] = 2022

marketing['month'] = marketing['month'].str.capitalize()
marketing['month'].unique()

marketing['date_str'] = marketing['year'].astype(str) + '-' + marketing['month'].astype(str) + '-' + marketing['day'].astype(str)

campaign['last_contact_date'] = pd.to_datetime(marketing['date_str'], format = '%Y-%b-%d', errors='coerce')

# Saving to CSV
client.to_csv('client.csv', index = False)
campaign.to_csv('campaign.csv', index = False)
companies.to_csv('economics.csv', index = False)

0        0
1        0
2        0
3        0
4        0
        ..
41183    0
41184    0
41185    0
41186    0
41187    0
Name: previous_outcome, Length: 41188, dtype: int64


In [7]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
